In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, results as rs,scsdata

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

"""path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'
report_start = pd.to_datetime('2021-20-01 06:00:00',dayfirst=True)
report_end = pd.to_datetime('2021-20-01 18:00:00',dayfirst=True)

at = pd.read_csv(path+'at_prod.csv')
av = pd.read_csv(path+'av_prod.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
fa = pd.read_csv(path + 'fa_dec_jan.csv')

# Create sc object from Class
sc = scsdata.ScsData('scs', av, at, fa)
# Call pre-process methods
sc.pre_process_at()
sc.pre_process_av()
sc.pre_process_faults()

output, R2, num_assets = rs.run_single_model(sc_data=sc, 
            report_start=report_start, report_end=report_end, shift=0, weights=[1], speed=470, 
            picker_present=0.91, availability=0.71)"""

"path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'\nreport_start = pd.to_datetime('2021-20-01 06:00:00',dayfirst=True)\nreport_end = pd.to_datetime('2021-20-01 18:00:00',dayfirst=True)\n\nat = pd.read_csv(path+'at_prod.csv')\nav = pd.read_csv(path+'av_prod.csv')\n#fa = pd.read_csv(path + 'scs_tote_matched.csv')\nfa = pd.read_csv(path + 'fa_dec_jan.csv')\n\n# Create sc object from Class\nsc = scsdata.ScsData('scs', av, at, fa)\n# Call pre-process methods\nsc.pre_process_at()\nsc.pre_process_av()\nsc.pre_process_faults()\n\noutput, R2, num_assets = rs.run_single_model(sc_data=sc, \n            report_start=report_start, report_end=report_end, shift=0, weights=[1], speed=470, \n            picker_present=0.91, availability=0.71)"

In [3]:
output

,NUMBER,ALERT,ENTRY_TIME,END_TIME,PLC,DESK,FAULT_ID,ID,AREA,BLUEGREY,PTT,SINGLES
976,42581.0,C05: C05PTT011_ external unit fault,2021-01-20 06:23:38,2021-01-20 06:23:53,C05,Z,3611.0,C05PTT011,PTT,Both,PTT011,3.697432
977,42581.0,C05: C05PTT011_ external unit fault,2021-01-20 07:01:27,2021-01-20 07:02:18,C05,Z,3611.0,C05PTT011,PTT,Both,PTT011,4.917703
978,42581.0,C05: C05PTT011_ external unit fault,2021-01-20 07:02:20,2021-01-20 07:02:40,C05,Z,3611.0,C05PTT011,PTT,Both,PTT011,3.984290
979,42581.0,C05: C05PTT011_ external unit fault,2021-01-20 07:31:55,2021-01-20 07:34:26,C05,Z,3611.0,C05PTT011,PTT,Both,PTT011,6.000049
980,42581.0,C05: C05PTT011_ external unit fault,2021-01-20 08:52:15,2021-01-20 08:52:48,C05,Z,3611.0,C05PTT011,PTT,Both,PTT011,4.483632
...,...,...,...,...,...,...,...,...,...,...,...,...
424301,131957.0,C16: C1603RDC143 143B11 occupancy sensor fault,2021-01-20 07:42:14,2021-01-20 07:42:26,C16,P03,3416.0,C1603RDC143,C16,Blue,PTT202,3.408992
424302,131957.0,C16: C1603RDC143 143B11 occupancy sensor fault,2021-01-20 07:27:11,2021-01-20 07:28:49,C16,P03,3416.0,C1603RDC143,C16,Blue,PTT202,5.463306
424303,131957.0,C16: C1603RDC143 143B11 occupancy sensor fault,2021-01-20 07:14:59,2021-01-20 07:16:16,C16,P03,3416.0,C1603RDC143,C16,Blue,PTT202,5.227397
424411,131494.0,C16: C1603ACH100 100B12 occupancy sensor fault,2021-01-20 13:05:27,2021-01-20 13:05:31,C16,P03,3416.0,C1603ACH100,C16,Blue,PTT202,0.912800


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [18]:
at = pd.read_csv(path+'at_prod.csv')
av = pd.read_csv(path+'av_prod.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'fa_dec_jan.csv')

In [19]:
at = feat.pre_process_at(at)

Running: Function "pre_process_at" (id=2099130373888) was called 2 times


In [20]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2099130372880) was called 2 times


In [21]:
fa,unmapped = feat.pre_process_fa(scs_raw)

#fa= fa[fa['timestamp']<(fa['timestamp'].max()-pd.to_timedelta(7, unit='D'))]

Running: Function "pre_process_fa" (id=2099130375040) was called 2 times
Running: Function "load_tote_lookup" (id=2099130375472) was called 2 times
Running: Function "load_module_lookup" (id=2099127532704) was called 2 times
Running: Function "load_ID_lookup" (id=2099130373456) was called 2 times


In [22]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Running: Function "floor_shift_time_fa" (id=2099130374464) was called 2 times


In [23]:
fa_floor[fa_floor['Entry Time']>"2021-01-20 17:00"]

,Alert ID,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Area,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type,Entry Time,End Time
238436,13303.0,114773.0,C14: C14PTT192_ external unit fault,2021-01-20 17:00:00,C14,Z,2.386294,3611,C14PTT192,PTT,Both,NaN,NaN,PTT192,14.0,4.0,19,Fault,2021-01-20 17:43:53,2021-01-20 17:43:57
238438,13314.0,132630.0,C16: C1601RCL211 211B51 sensor fault,2021-01-20 17:00:00,C16,P01,2.098612,3428,C1601RCL211,C16,Blue,NaN,NaN,False,16.0,0.0,NaN,Fault,2021-01-20 17:39:24,2021-01-20 17:39:27
238439,13355.0,500967.0,C37: emergency stop,2021-01-20 17:00:00,C37,Z,4.970292,0,C37,Stacker/Destacker,Blue,NaN,NaN,False,37.0,0.0,NaN,Fault,2021-01-20 17:30:53,2021-01-20 17:31:46
238440,13309.0,114773.0,C14: C14PTT192_ external unit fault,2021-01-20 17:00:00,C14,Z,2.609438,3611,C14PTT192,PTT,Both,NaN,NaN,PTT192,14.0,4.0,19,Fault,2021-01-20 17:41:03,2021-01-20 17:41:08
238441,13449.0,99481.0,C12: C1202STA082 082B22 occupancy sensor fault,2021-01-20 17:00:00,C12,P02,3.079442,3416,C1202STA082,Module Loop Grey,Grey,Module,C1202,False,12.0,3.0,14,Fault,2021-01-20 17:02:00,2021-01-20 17:02:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249754,0.0,188662.0,C23: C2301BCL177 177B51 sensor fault,2021-01-24 06:00:00,C23,P01,7.066108,3428,C2301BCL177,C23,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault,2021-01-24 06:00:00,2021-01-24 06:07:11
249755,13.0,406154.0,SCS044: sto: timeout bin from lifter to car X:...,2021-01-24 06:00:00,SCSM04,SCS044,8.220374,5105,SCS044,Carousel,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault,2021-01-24 06:00:00,2021-01-24 06:22:47
249756,1153.0,188345.0,C23: C2303BCL151 151B54 sensor fault,2021-01-24 07:00:00,C23,P03,9.188689,3428,C2303BCL151,C23,Blue,Outside,C2303,False,23.0,0.0,NaN,Fault,2021-01-24 07:00:00,2021-01-24 08:00:00
249757,47.0,132870.0,C16: C1601STA227 227B12 occupancy sensor fault,2021-01-24 07:00:00,C16,P01,6.424950,3416,C1601STA227,C16,Blue,NaN,NaN,False,16.0,0.0,NaN,Fault,2021-01-24 07:00:00,2021-01-24 07:03:47


In [24]:
df,fa_PTT = feat.create_PTT_df(fa_floor,at,av,weights = [1])

#df = feat.log_totes(df) 

Running: Function "create_PTT_df" (id=2099130448048) was called 2 times
Running: Function "fault_select" (id=2099130446032) was called 37 times
Running: Function "load_PTT_lookup" (id=2099130375904) was called 37 times
Running: Function "faults_aggregate" (id=2099130446320) was called 37 times
Running: Function "weight_hours" (id=2099130447472) was called 37 times
Running: Function "av_at_select" (id=2099130446608) was called 37 times
Running: Function "aggregate_availability" (id=2099130446896) was called 37 times
Running: Function "aggregate_totes" (id=2099130447184) was called 37 times
Running: Function "merge_av_fa_at" (id=2099130447760) was called 37 times
Running: Function "fault_select" (id=2099130446032) was called 38 times
Running: Function "load_PTT_lookup" (id=2099130375904) was called 38 times
Running: Function "faults_aggregate" (id=2099130446320) was called 38 times
Running: Function "weight_hours" (id=2099130447472) was called 38 times
Running: Function "av_at_select" (i

Running: Function "fault_select" (id=2099130446032) was called 51 times
Running: Function "load_PTT_lookup" (id=2099130375904) was called 51 times
Running: Function "faults_aggregate" (id=2099130446320) was called 51 times
Running: Function "weight_hours" (id=2099130447472) was called 51 times
Running: Function "av_at_select" (id=2099130446608) was called 51 times
Running: Function "aggregate_availability" (id=2099130446896) was called 51 times
Running: Function "aggregate_totes" (id=2099130447184) was called 51 times
Running: Function "merge_av_fa_at" (id=2099130447760) was called 51 times
Running: Function "fault_select" (id=2099130446032) was called 52 times
Running: Function "load_PTT_lookup" (id=2099130375904) was called 52 times
Running: Function "faults_aggregate" (id=2099130446320) was called 52 times
Running: Function "weight_hours" (id=2099130447472) was called 52 times
Running: Function "av_at_select" (id=2099130446608) was called 52 times
Running: Function "aggregate_availa

Running: Function "fault_select" (id=2099130446032) was called 65 times
Running: Function "load_PTT_lookup" (id=2099130375904) was called 65 times
Running: Function "faults_aggregate" (id=2099130446320) was called 65 times
Running: Function "weight_hours" (id=2099130447472) was called 65 times
Running: Function "av_at_select" (id=2099130446608) was called 65 times
Running: Function "aggregate_availability" (id=2099130446896) was called 65 times
Running: Function "aggregate_totes" (id=2099130447184) was called 65 times
Running: Function "merge_av_fa_at" (id=2099130447760) was called 65 times
Running: Function "fault_select" (id=2099130446032) was called 66 times
Running: Function "load_PTT_lookup" (id=2099130375904) was called 66 times
Running: Function "faults_aggregate" (id=2099130446320) was called 66 times
Running: Function "weight_hours" (id=2099130447472) was called 66 times
Running: Function "av_at_select" (id=2099130446608) was called 66 times
Running: Function "aggregate_availa

In [17]:
fa_PTT['PTT011']

,Alert ID,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Area,Tote Colour,Loop,Suffix,PLCN,Quadrant,MODULE,Alert Type,Entry Time,End Time,Pick Station
77271,191528.0,42581.0,C05: C05PTT011_ external unit fault,2020-11-20 00:00:00,C05,Z,4.828641,3611.0,C05PTT011,PTT,Both,NaN,NaN,5.0,1.0,1,Fault,2020-11-20 00:32:00,2020-11-20 00:32:46,PTT011
77272,190344.0,42581.0,C05: C05PTT011_ external unit fault,2020-11-20 03:00:00,C05,Z,2.791759,3611.0,C05PTT011,PTT,Both,NaN,NaN,5.0,1.0,1,Fault,2020-11-20 03:46:00,2020-11-20 03:46:06,PTT011
77273,190141.0,42581.0,C05: C05PTT011_ external unit fault,2020-11-20 04:00:00,C05,Z,2.609438,3611.0,C05PTT011,PTT,Both,NaN,NaN,5.0,1.0,1,Fault,2020-11-20 04:25:00,2020-11-20 04:25:05,PTT011
77274,189201.0,42581.0,C05: C05PTT011_ external unit fault,2020-11-20 07:00:00,C05,Z,4.663562,3611.0,C05PTT011,PTT,Both,NaN,NaN,5.0,1.0,1,Fault,2020-11-20 07:46:00,2020-11-20 07:46:39,PTT011
77275,189058.0,42581.0,C05: C05PTT011_ external unit fault,2020-11-20 08:00:00,C05,Z,2.609438,3611.0,C05PTT011,PTT,Both,NaN,NaN,5.0,1.0,1,Fault,2020-11-20 08:18:00,2020-11-20 08:18:05,PTT011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249664,19345.0,123351.0,C15: C1501ACH087 087QM11 motor circuit breaker...,2021-01-19 06:00:00,C15,P01,6.605802,3430.0,C1501ACH087,C15,Blue,Outside,C1501,15.0,0.0,NaN,Fault,2021-01-19 06:26:51,2021-01-19 06:31:23,False
249692,15091.0,122423.0,C15: C1502STA028 028B21 occupancy sensor fault,2021-01-20 09:00:00,C15,P02,2.609438,3416.0,C1502STA028,C15,Blue,Outside,C1502,15.0,0.0,NaN,Fault,2021-01-20 09:55:53,2021-01-20 09:55:58,False
249704,13896.0,123189.0,C15: C1502RDC077 077B11 occupancy sensor fault,2021-01-20 14:00:00,C15,P02,5.094345,3416.0,C1502RDC077,C15,Blue,Outside,C1502,15.0,0.0,NaN,Fault,2021-01-20 14:59:00,2021-01-20 15:00:00,False
249705,13896.0,123189.0,C15: C1502RDC077 077B11 occupancy sensor fault,2021-01-20 15:00:00,C15,P02,7.188264,3416.0,C1502RDC077,C15,Blue,Outside,C1502,15.0,0.0,NaN,Fault,2021-01-20 15:00:00,2021-01-20 15:08:07,False


In [ ]:
df_week = df[df['timestamp']>end_time - pd.to_timedelta(14, unit='D')]

In [ ]:
X,y = md.gen_feat_var(df,target = 'Availability', features = ['Faults','Totes'])


In [ ]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': 42})

R2_cv,R2_OOS,Coeff = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 30)

In [ ]:
R2_OOS

# Plotting

In [ ]:
Output[Output['AREA'] == 'PTT']['ID'].value_counts()

In [ ]:
Total = Output[['ID','SINGLES']].groupby('ID').sum().reset_index()

In [ ]:
plt.figure(figsize=(15,5))
sns.barplot(y='SINGLES',x = 'ID',data = Total.sort_values(by='SINGLES',ascending=False).head(10),color = 'r')
plt.ylabel('Lost Singles')
plt.title('SCS Lost Singles Last 12 Hours')

In [ ]:
PTT081 = Output[['Asset Code','PTT081']].groupby('Asset Code').sum().reset_index()

In [ ]:
plt.figure(figsize=(7,5))
sns.barplot(y='PTT081',x = 'Asset Code',data = PTT081.sort_values(by='PTT081',ascending=False).head(5),color = 'r')
plt.ylabel('Lost Singles')
plt.title('PTT081 Lost Singles Last 12 Hours')

In [ ]:
Total_colour = Output[['BLUEGREY','SINGLES']].groupby('BLUEGREY').sum().reset_index()

In [ ]:
plt.figure(figsize=(5,5))
sns.barplot(y='SINGLES',x = 'BLUEGREY',data = Total_colour.sort_values(by='SINGLES',ascending=False),color = 'r')
plt.ylabel('Lost Singles')
plt.title('SCS Lost Singles Last 12 Hours')

In [ ]:
Total_colour

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df, line_kws={"color": "red"})
plt.xlim(0, 5)

In [ ]:
sns.lmplot(x='TOTES', y='Availability', data=df_week, line_kws={"color": "red"})

In [ ]:
ax = sns.regplot(x="TOTES", y="Availability", data=df_week,
x_estimator=np.mean, logx=True, line_kws={"color": "red"})
plt.ylim(0,1)

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df_week, line_kws={"color": "red"})
plt.xlim(0, 5)

In [ ]:
Shift = [0,0,0,10,10,10,20,20,20]
Weights = [[1],[0.7,0.3],[0.7,0.2,0.1],[1],[0.7,0.3],[0.7,0.2,0.1],[1],[0.7,0.3],[0.7,0.2,0.1]]

In [ ]:
Weights[8]

In [ ]:
y

In [ ]:
at

In [ ]:
fa_floor

In [ ]:
df.drop(['Availability','Blue Tote Loss','Grey Tote Loss','timestamp'],axis=1)

In [ ]:
Outputs = {0.5:'Number One',0.4:'Number Two',-1.4:'Number 3'}

In [ ]:
 Output = Outputs[max(k for k, v in Outputs.items())]

In [ ]:
Outputs[0.456]='Numbers 4'

In [ ]:
Outputs

In [ ]:
print('Selected R2:', max(k for k, v in Outputs.items()))

In [ ]:
fa_floor = feat.floor_shift_time_fa(fa, shift=10)

df,fa_PTT = feat.create_PTT_df(fa_floor,at,av,weights=[0.7,0.2,0.1])
df = feat.log_totes(df) 
df_2week = df[df['timestamp']>end_time - pd.to_timedelta(14, unit='D')]
X,y = md.gen_feat_var(df_2week,target = 'Availability', features = ['Totes','Faults'])
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                    'random_state': None})

R2_cv,R2_OOS,Coeff = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 30)

Output = rs.create_output(fa_PTT,Coeff,end_time,speed = speed, picker_present = picker_present, availability = availability)

In [ ]:
R2_OOS

In [ ]:
fa_floor

In [ ]:
print(fa.loc[fa['Area'].isnull() & fa['Desk'] == 'Z', 'Area'])

In [ ]:
fa['Area'].isnull() & fa['Desk'] == 'Z' 

In [ ]:
fa

In [ ]:
type(fa)

In [ ]:
from datetime import datetime 
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

In [ ]:
log = pd.read_excel('C:/Users/Jamie.williams/desktop/scs/Run_log.xlsx')
new_row = pd.DataFrame([['dt_string','R2_sel','feat_sel','runtime','report_start','report_end']],columns = list(log.columns))

In [ ]:
new_row

In [ ]:
import time
now = datetime.now()
time.sleep(2)
now2 = datetime.now()
now2-now

In [ ]:
'asdsad' +'asddasd'

In [ ]:
fa['timestamp'].max()

In [ ]:
max_date = pd.to_datetime(scs_raw['Entry Time'],dayfirst=True).max()

In [ ]:
max_date2 = pd.to_datetime(fa_old['Entry Time'],dayfirst=True).max()

In [16]:
Test_time = pd.to_timedelta(48,unit='H')

In [17]:
Test_time.days

2

In [49]:
av = pd.DataFrame({'timestamp':['2021-01-09 08:00:00','2021-01-10 08:00:00','2021-01-11 08:00:00','2021-01-12 08:00:00','2021-01-13 08:00:00'],'Pick Station':['PTT011','PTT051','PTT072','PTT112','PTT202'],'Availability':[0.9,0.8,0.75,0.3,0.1]})

In [50]:
av

,timestamp,Pick Station,Availability
0,2021-01-09 08:00:00,PTT011,0.90
1,2021-01-10 08:00:00,PTT051,0.80
2,2021-01-11 08:00:00,PTT072,0.75
3,2021-01-12 08:00:00,PTT112,0.30
4,2021-01-13 08:00:00,PTT202,0.10


In [12]:
fa

,Alert ID,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Area,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
260656,185605.0,186939.0,C23: C2301RDC061 061AQ1 roller motor unit fault,2020-11-20 20:21:00,C23,P01,1.0,3429,C2301RDC061,C23,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault
87785,273819.0,50773.0,C06: C06PTT022_ external unit fault,2020-12-02 03:24:00,C06,Z,1.0,3611,C06PTT022,PTT,Both,NaN,NaN,PTT022,6.0,1.0,2,Fault
260657,185607.0,186939.0,C23: C2301RDC061 061AQ1 roller motor unit fault,2020-11-20 20:20:00,C23,P01,1.0,3429,C2301RDC061,C23,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault
20962,218904.0,416682.0,SCS103: sto: timeout bin from lifter to car X:...,2020-11-28 18:46:00,SCSM10,SCS103,1.0,5105,SCS103,Carousel,Grey,carousel,SCSM10,False,0.0,2.0,10,Fault
260658,185614.0,186939.0,C23: C2301RDC061 061AQ1 roller motor unit fault,2020-11-20 20:19:00,C23,P01,1.0,3429,C2301RDC061,C23,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252406,140924.0,130631.0,C16: C1602ACL040 040QM11 motor circuit breaker...,2020-12-28 21:48:06,C16,P02,81929.0,3430,C1602ACL040,C16,Blue,NaN,NaN,False,16.0,0.0,NaN,Fault
259324,151857.0,42600.0,C05: C0502RDC038 038AQ1 roller motor unit fault,2020-12-25 04:56:43,C05,P02,82717.0,3429,C0502RDC038,Module Loop Blue,Blue,Module,C0502,False,5.0,1.0,1,Fault
1044,79527.0,188345.0,C23: C2303BCL151 151B54 sensor fault,2021-01-07 07:58:47,C23,P03,82898.0,3428,C2303BCL151,C23,Blue,Outside,C2303,False,23.0,0.0,NaN,Fault
322041,79549.0,418195.0,SCS112: carousel: inverter error No.:14 [6.2],2021-01-07 07:48:37,SCSM11,SCS112,86222.0,5050,SCS112,Carousel,Grey,carousel,SCSM11,False,0.0,3.0,11,Fault


In [2]:
fa = pd.DataFrame({'Number': ['1','2','3','4','5',
                                    '6','7','8','9','10'],
                           'Alert': ['SCS116: sto: timeout bin from lifter to car X:5 Y:4 S:2 [13.1] ',
                                    'C05: C05PTT011_ external unit fault',
                                    'C16: C1603STA253 253B12 occupancy sensor fault',
                                    'C15: C1502RDC070 070B11 occupancy sensor fault',
                                    'C23: C2301RDC176 176B14 occupancy sensor fault',
                                    'SCS M20: single module 1 emergency stop unit     [24.0]',
                                    'C14: C1404STA164 164D702 scanner / data tracking fault',
                                    'C07: C0702STA074 074D702 scanner / data tracking fault',
                                    'C08: C0803STA102 102D702 scanner / data tracking fault',
                                    'C10: C1002STA081 081B15 clearance sensor fault'],
                            'Entry Time': ['2021-01-12 08:12:00',
                                         '2021-01-09 08:56:00',
                                         '2021-01-11 08:34:00',
                                         '2021-01-09 08:20:00',
                                         '2021-01-10 07:59:00',
                                         '2021-01-13 08:14:00',
                                         '2021-01-13 08:46:00',
                                         '2021-01-10 08:49:00',
                                         '2021-01-11 08:11:00',
                                         '2021-01-12 08:01:00'],
                            'PLC': ['SCSM11','C05','C16','C15','C23',
                                    'SCSM20','C14','C07','C08','C10'],
                            'Desk': ['SCS116','Z','P03','P02','P01',
                                    'SCSM20','P04','P02','P03','P02'],
                            'Duration': ['0.00:00:30',
                                         '0.00:05:30',
                                         '0.00:07:20',
                                         '0.00:15:30',
                                         '0.00:31:30',
                                         '0.01:12:30',
                                         '0.00:00:05',
                                         '0.00:00:07',
                                         '0.00:00:01',
                                         '0.00:00:03'],
                            'Fault ID': ['5105','3611','3416','3416','3416',
                                    '5416','3504','3504','3504','3419'],
                            'Asset Code': ['SCS116','C05','C1603STA253','C1502RDC070',
                                          'C2301RDC176','SCSM20','C1404STA164','C0702STA074',
                                          'C0803STA102','C1002STA081']})

In [3]:
fa_processed,unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup"                 (id=2299071606496) was called 1 times


In [39]:
fa_processed

,Number,Alert,Entry Time,PLC,Desk,Duration,Fault ID,Asset Code,Area,Tote Colour,Loop,Suffix,Pick Station,PLCN
0,1,SCS116: sto: timeout bin from lifter to car X:...,2021-01-12 08:12:00,SCSM11,SCS116,0.00:00:30,5105,SCS116,Carousel,Grey,carousel,SCSM11,False,0
1,2,C05: C05PTT011_ external unit fault,2021-01-09 08:56:00,C05,Z,0.00:05:30,3611,C05,PTT,Both,NaN,NaN,PTT011,5
2,3,C16: C1603STA253 253B12 occupancy sensor fault,2021-01-11 08:34:00,C16,P03,0.00:07:20,3416,C1603STA253,C16,Blue,Outside,C1603,False,16
3,4,C15: C1502RDC070 070B11 occupancy sensor fault,2021-01-09 08:20:00,C15,P02,0.00:15:30,3416,C1502RDC070,C15,Blue,Outside,C1502,False,15
4,5,C23: C2301RDC176 176B14 occupancy sensor fault,2021-01-10 07:59:00,C23,P01,0.00:31:30,3416,C2301RDC176,C23,Blue,Outside,C2301,False,23
5,6,SCS M20: single module 1 emergency stop unit ...,2021-01-13 08:14:00,SCSM20,SCSM20,0.01:12:30,5416,SCSM20,Carousel,Grey,carousel,SCSM20,False,0
6,7,C14: C1404STA164 164D702 scanner / data tracki...,2021-01-13 08:46:00,C14,P04,0.00:00:05,3504,C1404STA164,Module Loop Grey,Grey,Module,C1404,False,14
7,8,C07: C0702STA074 074D702 scanner / data tracki...,2021-01-10 08:49:00,C07,P02,0.00:00:07,3504,C0702STA074,Module Loop Grey,Grey,Module,C0702,False,7
8,9,C08: C0803STA102 102D702 scanner / data tracki...,2021-01-11 08:11:00,C08,P03,0.00:00:01,3504,C0803STA102,Quadrant Loop Blue,Blue,Quadrant,C0803,False,8
9,10,C10: C1002STA081 081B15 clearance sensor fault,2021-01-12 08:01:00,C10,P02,0.00:00:03,3419,C1002STA081,Module Loop Grey,Grey,Module,C1002,False,10


In [40]:
fa_processed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Number        10 non-null     object
 1   Alert         10 non-null     object
 2   Entry Time    10 non-null     object
 3   PLC           10 non-null     object
 4   Desk          10 non-null     object
 5   Duration      10 non-null     object
 6   Fault ID      10 non-null     object
 7   Asset Code    10 non-null     object
 8   Area          10 non-null     object
 9   Tote Colour   10 non-null     object
 10  Loop          9 non-null      object
 11  Suffix        9 non-null      object
 12  Pick Station  10 non-null     object
 13  PLCN          10 non-null     int32 
dtypes: int32(1), object(13)
memory usage: 1.1+ KB


In [11]:
fa = pd.DataFrame({'Number': ['1','2','3','4','5',
                                    '6','7','8','9','10'],
                           'Alert': ['SCS116: sto: timeout bin from lifter to car X:5 Y:4 S:2 [13.1] ',
                                    'C05: C05PTT011_ external unit fault',
                                    'C16: C1603STA253 253B12 occupancy sensor fault',
                                    'C15: C1502RDC070 070B11 occupancy sensor fault',
                                    'C23: C2301RDC176 176B14 occupancy sensor fault',
                                    'SCS M20: single module 1 emergency stop unit     [24.0]',
                                    'C14: C1402STA049 049B14 clearance sensor fault',
                                    'C07: C0702STA049 049B14 clearance sensor fault',
                                    'C08: C0803STA102 102D702 scanner / data tracking fault',
                                    'C10: C1002STA081 081B15 clearance sensor fault'],
                            'Entry Time': ['2021-01-12 08:12:00',
                                         '2021-01-09 08:56:00',
                                         '2021-01-11 08:34:00',
                                         '2021-01-09 08:20:00',
                                         '2021-01-10 07:59:00',
                                         '2021-01-13 08:14:00',
                                         '2021-01-13 08:46:00',
                                         '2021-01-10 08:49:00',
                                         '2021-01-11 08:11:00',
                                         '2021-01-12 08:01:00'],
                            'PLC': ['SCSM11','C05','C16','C15','C23',
                                    'SCSM20','C14','C07','C08','C10'],
                            'Desk': ['SCS116','Z','P03','P02','P01',
                                    'SCSM20','P02','P02','P03','P02'],
                            'Duration': ['0.00:00:30',
                                         '0.00:05:30',
                                         '0.00:07:20',
                                         '0.00:15:30',
                                         '0.00:31:30',
                                         '0.01:12:30',
                                         '0.00:00:05',
                                         '0.00:00:07',
                                         '0.00:00:01',
                                         '0.00:00:03'],
                            'Fault ID': [5105,3611,3416,3416,3416,
                                    5416,3419,3419,3504,3419]})    


fa_processed,unmapped = feat.pre_process_fa(fa)

Running: Function "pre_process_fa"                 (id=2299071604912) was called 4 times
Running: Function "load_tote_lookup"                 (id=2299071606496) was called 5 times
Running: Function "load_module_lookup"                 (id=2299068628416) was called 4 times
Running: Function "load_ID_lookup"                 (id=2299071605632) was called 4 times
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Alert ID      10 non-null     int64         
 1   Number        10 non-null     object        
 2   Alert         10 non-null     object        
 3   timestamp     10 non-null     datetime64[ns]
 4   PLC           10 non-null     object        
 5   Desk          10 non-null     object        
 6   Duration      10 non-null     object        
 7   Fault ID      10 non-null     int64         
 8   Asset Code    10 non-null     o

In [12]:
fa_processed

,Alert ID,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Area,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
8,9.0,10,C10: C1002STA081 081B15 clearance sensor fault,2021-01-12 08:01:00,C10,P02,3.0,3419,C1002STA081,Module Loop Grey,Grey,Module,C1002,False,10.0,3.0,11,Fault
6,6.0,7,C14: C1402STA049 049B14 clearance sensor fault,2021-01-13 08:46:00,C14,P02,5.0,3419,C1402STA049,Module Loop Blue,Blue,Module,C1402,False,14.0,4.0,19,Fault
7,7.0,8,C07: C0702STA049 049B14 clearance sensor fault,2021-01-10 08:49:00,C07,P02,7.0,3419,C0702STA049,Module Loop Blue,Blue,Module,C0702,False,7.0,1.0,4,Fault
0,0.0,1,SCS116: sto: timeout bin from lifter to car X:...,2021-01-12 08:12:00,SCSM11,SCS116,30.0,5105,SCS116,Carousel,Grey,carousel,SCSM11,False,0.0,3.0,11,Fault
1,1.0,2,C05: C05PTT011_ external unit fault,2021-01-09 08:56:00,C05,Z,330.0,3611,C05PTT011,PTT,Both,NaN,NaN,PTT011,5.0,1.0,1,Fault
2,2.0,3,C16: C1603STA253 253B12 occupancy sensor fault,2021-01-11 08:34:00,C16,P03,440.0,3416,C1603STA253,C16,Blue,Outside,C1603,False,16.0,0.0,NaN,Fault
3,3.0,4,C15: C1502RDC070 070B11 occupancy sensor fault,2021-01-09 08:20:00,C15,P02,930.0,3416,C1502RDC070,C15,Blue,Outside,C1502,False,15.0,0.0,NaN,Fault
4,4.0,5,C23: C2301RDC176 176B14 occupancy sensor fault,2021-01-10 07:59:00,C23,P01,1890.0,3416,C2301RDC176,C23,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault
5,5.0,6,SCS M20: single module 1 emergency stop unit ...,2021-01-13 08:14:00,SCSM20,SCSM20,4350.0,5416,SCSM20,Carousel,Grey,carousel,SCSM20,False,0.0,4.0,20,Fault


In [14]:
fa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Number      10 non-null     object
 1   Alert       10 non-null     object
 2   Entry Time  10 non-null     object
 3   PLC         10 non-null     object
 4   Desk        10 non-null     object
 5   Duration    10 non-null     object
 6   Fault ID    10 non-null     int64 
dtypes: int64(1), object(6)
memory usage: 688.0+ bytes
